In [1]:
from netCDF4 import Dataset
import numpy as np
import os,time
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

ModuleNotFoundError: No module named 'netCDF4'

设置

In [2]:
path_in = '/work/wanbc/data_folder/python/NJ_radar_HD_REF/'

In [3]:
path_out = '/work/wuxue/data/CI/JS/'

In [4]:
time_start = '201806140420'
time_end = '201806141030'

In [5]:
lon_start = 116
lon_end = 122
lat_start = 30
lat_end = 36

In [6]:
cmpdata=np.loadtxt('./radar.rgb')
cmpdata=cmpdata/255.0;
cmpdata_35=np.loadtxt('./radar_35.rgb')
cmpdata_35=cmpdata_35/255.0;

In [7]:
n = 5                      # 取周围格点范围

时空范围

In [8]:
time_s = time.mktime(time.strptime(time_start,'%Y%m%d%H%M'))-60*10*3    # CI判断前3个时次
time_e = time.mktime(time.strptime(time_end,'%Y%m%d%H%M'))
times = np.arange(time_s,time_e+1,10*60)
# times = np.arange(time_s,time_s+60*60*2+1,10*60)

In [9]:
lon_s = 300                                                 ## list(lon).index(lon_start)
lon_e = 600                                                 ## list(lon).index(lon_end)
lat_s = 250                                                 ## list(lat).index(lat_start)
lat_e = 550                                                 ## list(lat).index(lat_end)

for

In [10]:
for t in range(3,len(times)):
    time_0 = time.strftime("%Y%m%d%H%M%S", time.localtime(times[t]))
    time_1 = time.strftime("%Y%m%d%H%M%S", time.localtime(times[t-1]))
    time_2 = time.strftime("%Y%m%d%H%M%S", time.localtime(times[t-2]))
    time_3 = time.strftime("%Y%m%d%H%M%S", time.localtime(times[t-3]))
    print(time_0)

    radar_file_0 = os.path.join(path_in, time_0[:8], 'OBS_REF_'+time_0+'.nc')
    radar_file_1 = os.path.join(path_in, time_1[:8], 'OBS_REF_'+time_1+'.nc')
    radar_file_2 = os.path.join(path_in, time_2[:8], 'OBS_REF_'+time_2+'.nc')
    radar_file_3 = os.path.join(path_in, time_3[:8], 'OBS_REF_'+time_3+'.nc')    
    try:
        radar_0 = Dataset(radar_file_0) 
        radar_1 = Dataset(radar_file_1) 
        radar_2 = Dataset(radar_file_2)
        radar_3 = Dataset(radar_file_3)   
    except:
        print('No Radar files!')
        continue
    lat = radar_0.variables['lat'][lat_s:lat_e+1].data
    lon = radar_0.variables['lon'][lon_s:lon_e+1].data
    REF0 = radar_0.variables['REF'][lat_s:lat_e+1,lon_s:lon_e+1].data
    REF1 = radar_1.variables['REF'][lat_s:lat_e+1,lon_s:lon_e+1].data
    REF2 = radar_2.variables['REF'][lat_s:lat_e+1,lon_s:lon_e+1].data
    REF3 = radar_3.variables['REF'][lat_s:lat_e+1,lon_s:lon_e+1].data
    REF0[REF0<0]=0
    REF1[REF1<0]=0
    
    CI = np.zeros_like(REF0)
    for y in range(0+n,len(lat)-n-1):
        for x in range(0+n,len(lon)-n-1):
            rd1 = REF1[y-n:y+n+1,x-n:x+n+1]              # 取周围格点
            rd2 = REF2[y-n:y+n+1,x-n:x+n+1]
            rd3 = REF3[y-n:y+n+1,x-n:x+n+1]
            n1 = (rd1>=35).sum()                         # 周围格点中大于35dBZ的格点数
            n2 = (rd2>=35).sum()
            n3 = (rd3>=35).sum()
            # 当前格点达到35dBZ，且周围没有（不是从周围移来或分裂的）
            if (REF0[y,x]>=35) and n1==0 and n2==0 :       # and n3==0 
                CI[y,x]=1
    n_CI = CI[CI==1].sum()
                
    CI_path = os.path.join(path_out, time_0[:8])
    try:
        os.makedirs(CI_path)
    except:
        pass
    CI_file = os.path.join(CI_path, 'CI_'+time_0+'.nc')
    REF_image = os.path.join(CI_path, 'REF_'+time_0+'.png')
    CI_image = os.path.join(CI_path, 'REF_CI_'+time_0+'.png')
    CI_35_image = os.path.join(CI_path, 'REF_35_CI_'+time_0+'.png')
    
    # data2nc
    da=Dataset(CI_file,"w",format="NETCDF4")
    da.createDimension("latsize",len(lat))
    da.createDimension("lonsize",len(lon))
    da.createVariable("CI","f8",("latsize","lonsize"))
    da.createVariable("lon","f4",("latsize"))
    da.createVariable("lat","f4",("lonsize"))
    da.variables["CI"][:]=CI[:]
    da.variables["lat"][:]=lat[:]
    da.variables["lon"][:]=lon[:]
    da.description="radar reflectivity"
    da.close()    
    print('CI data saved as '+ CI_file)   
    
    # REF
    fig = plt.figure(figsize=(16,16))
    map=Basemap(projection='cyl',resolution='i',llcrnrlon=lon_start,llcrnrlat=lat_start,urcrnrlon=lon_end,urcrnrlat=lat_end,suppress_ticks=False)
    map.drawcoastlines(linewidth=1,color='k')
    map.drawcountries(linewidth=1,color='k')
    map.drawmeridians(np.arange(lon_start,lon_end,1))
    map.drawparallels(np.arange(lat_start,lat_end,1))
    levels = [0,5,10,15,20,25,30,35,40,45,50,55,60]
    sc=plt.contourf(lon,lat,REF0,levels=levels,colors=cmpdata,vmin=0,vmax=80)
    plt.colorbar(orientation='vertical',fraction=0.04, ticks=levels,label='dBZ')       
    plt.title('REF '+time_0+'(UTC)',fontsize = 20)     
    plt.savefig(REF_image, bbox_inches='tight') 
#     plt.show()
    plt.close()     
    print('REF image saved as '+ REF_image)
    
    # REF-CI
    fig = plt.figure(figsize=(48,16))
    ax1 = fig.add_subplot(1,3,1)
    ax2 = fig.add_subplot(1,3,2)
    ax3 = fig.add_subplot(1,3,3)
    plt.sca(ax1)            # -10min REF
    map=Basemap(projection='cyl',resolution='i',llcrnrlon=lon_start,llcrnrlat=lat_start,urcrnrlon=lon_end,urcrnrlat=lat_end,suppress_ticks=False)
    map.drawcoastlines(linewidth=1,color='k')
    map.drawcountries(linewidth=1,color='k')
    map.drawmeridians(np.arange(lon_start,lon_end,1))
    map.drawparallels(np.arange(lat_start,lat_end,1))
    levels = [0,5,10,15,20,25,30,35,40,45,50,55,60]
    sc=plt.contourf(lon,lat,REF1,levels=levels,colors=cmpdata,vmin=0,vmax=80)
    plt.colorbar(orientation='vertical',fraction=0.04, ticks=levels,label='dBZ')       
    plt.title('REF '+time_1+'(UTC)',fontsize = 30)     
    plt.sca(ax2)          # REF
    map=Basemap(projection='cyl',resolution='i',llcrnrlon=lon_start,llcrnrlat=lat_start,urcrnrlon=lon_end,urcrnrlat=lat_end,suppress_ticks=False)
    map.drawcoastlines(linewidth=1,color='k')
    map.drawcountries(linewidth=1,color='k')
    map.drawmeridians(np.arange(lon_start,lon_end,1))
    map.drawparallels(np.arange(lat_start,lat_end,1))
    levels = [0,5,10,15,20,25,30,35,40,45,50,55,60]
    sc=plt.contourf(lon,lat,REF0,levels=levels,colors=cmpdata,vmin=0,vmax=80)
    plt.colorbar(orientation='vertical',fraction=0.04, ticks=levels,label='dBZ')       
    plt.title('REF '+time_0+'(UTC)',fontsize = 30) 
    plt.sca(ax3)           # CI
    map=Basemap(projection='cyl',resolution='i',llcrnrlon=lon_start,llcrnrlat=lat_start,urcrnrlon=lon_end,urcrnrlat=lat_end,suppress_ticks=False)
    map.drawcoastlines(linewidth=1,color='k')
    map.drawcountries(linewidth=1,color='k')
    map.drawmeridians(np.arange(lon_start,lon_end,1))
    map.drawparallels(np.arange(lat_start,lat_end,1))
    sc=plt.contourf(lon,lat,CI,1,colors=[[1,1,1],[1,0,0]],vmin=0,vmax=1)     
    plt.colorbar(orientation='vertical',fraction=0.04, ticks=[0,1],label='0-False                                          1-True') 
    plt.title('CI '+time_0+'(UTC)    CI grads: '+str(int(n_CI)),fontsize = 30) 
    plt.savefig(CI_image, bbox_inches='tight') 
    print('REF-CI image saved as '+ CI_image)
#     plt.show()
    plt.close()     
    
    # REF35-CI
    fig = plt.figure(figsize=(48,16))
    ax1 = fig.add_subplot(1,3,1)
    ax2 = fig.add_subplot(1,3,2)
    ax3 = fig.add_subplot(1,3,3)
    plt.sca(ax1)            # -10min REF
    map=Basemap(projection='cyl',resolution='i',llcrnrlon=lon_start,llcrnrlat=lat_start,urcrnrlon=lon_end,urcrnrlat=lat_end,suppress_ticks=False)
    map.drawcoastlines(linewidth=1,color='k')
    map.drawcountries(linewidth=1,color='k')
    map.drawmeridians(np.arange(lon_start,lon_end,1))
    map.drawparallels(np.arange(lat_start,lat_end,1))
    levels = [0,5,10,15,20,25,30,35,40,45,50,55,60]
    sc=plt.contourf(lon,lat,REF1,levels=levels,colors=cmpdata_35,vmin=0,vmax=80)
    plt.colorbar(orientation='vertical',fraction=0.04, ticks=levels,label='dBZ')       
    plt.title('REF '+time_1+'(UTC)',fontsize = 30)     
    plt.sca(ax2)            # REF
    map=Basemap(projection='cyl',resolution='i',llcrnrlon=lon_start,llcrnrlat=lat_start,urcrnrlon=lon_end,urcrnrlat=lat_end,suppress_ticks=False)
    map.drawcoastlines(linewidth=1,color='k')
    map.drawcountries(linewidth=1,color='k')
    map.drawmeridians(np.arange(lon_start,lon_end,1))
    map.drawparallels(np.arange(lat_start,lat_end,1))
    levels = [0,5,10,15,20,25,30,35,40,45,50,55,60]
    sc=plt.contourf(lon,lat,REF0,levels=levels,colors=cmpdata_35,vmin=0,vmax=80)
    plt.colorbar(orientation='vertical',fraction=0.04, ticks=levels,label='dBZ')       
    plt.title('REF '+time_0+'(UTC)',fontsize = 30)   
    plt.sca(ax3)             # CI
    map=Basemap(projection='cyl',resolution='i',llcrnrlon=lon_start,llcrnrlat=lat_start,urcrnrlon=lon_end,urcrnrlat=lat_end,suppress_ticks=False)
    map.drawcoastlines(linewidth=1,color='k')
    map.drawcountries(linewidth=1,color='k')
    map.drawmeridians(np.arange(lon_start,lon_end,1))
    map.drawparallels(np.arange(lat_start,lat_end,1))
    sc=plt.contourf(lon,lat,CI,1,colors=[[1,1,1],[1,0,0]],vmin=0,vmax=1)     
    plt.colorbar(orientation='vertical',fraction=0.04, ticks=[0,1],label='0-False                                          1-True') 
    plt.title('CI '+time_0+'(UTC)    CI grads: '+str(int(n_CI)),fontsize = 30) 
    plt.savefig(CI_35_image, bbox_inches='tight') 
    print('REF35-CI image saved as '+ CI_35_image)
#     plt.show()
    plt.close()

20180614042000
CI data saved as /work/wuxue/data/CI/JS/20180614/CI_20180614042000.nc
REF image saved as /work/wuxue/data/CI/JS/20180614/REF_20180614042000.png
REF-CI image saved as /work/wuxue/data/CI/JS/20180614/REF_CI_20180614042000.png
REF35-CI image saved as /work/wuxue/data/CI/JS/20180614/REF_35_CI_20180614042000.png
20180614043000
CI data saved as /work/wuxue/data/CI/JS/20180614/CI_20180614043000.nc
REF image saved as /work/wuxue/data/CI/JS/20180614/REF_20180614043000.png
REF-CI image saved as /work/wuxue/data/CI/JS/20180614/REF_CI_20180614043000.png
REF35-CI image saved as /work/wuxue/data/CI/JS/20180614/REF_35_CI_20180614043000.png
20180614044000
CI data saved as /work/wuxue/data/CI/JS/20180614/CI_20180614044000.nc
REF image saved as /work/wuxue/data/CI/JS/20180614/REF_20180614044000.png
REF-CI image saved as /work/wuxue/data/CI/JS/20180614/REF_CI_20180614044000.png
REF35-CI image saved as /work/wuxue/data/CI/JS/20180614/REF_35_CI_20180614044000.png
20180614045000
CI data saved

REF-CI image saved as /work/wuxue/data/CI/JS/20180614/REF_CI_20180614083000.png
REF35-CI image saved as /work/wuxue/data/CI/JS/20180614/REF_35_CI_20180614083000.png
20180614084000
CI data saved as /work/wuxue/data/CI/JS/20180614/CI_20180614084000.nc
REF image saved as /work/wuxue/data/CI/JS/20180614/REF_20180614084000.png
REF-CI image saved as /work/wuxue/data/CI/JS/20180614/REF_CI_20180614084000.png
REF35-CI image saved as /work/wuxue/data/CI/JS/20180614/REF_35_CI_20180614084000.png
20180614085000
CI data saved as /work/wuxue/data/CI/JS/20180614/CI_20180614085000.nc
REF image saved as /work/wuxue/data/CI/JS/20180614/REF_20180614085000.png
REF-CI image saved as /work/wuxue/data/CI/JS/20180614/REF_CI_20180614085000.png
REF35-CI image saved as /work/wuxue/data/CI/JS/20180614/REF_35_CI_20180614085000.png
20180614090000
CI data saved as /work/wuxue/data/CI/JS/20180614/CI_20180614090000.nc
REF image saved as /work/wuxue/data/CI/JS/20180614/REF_20180614090000.png
REF-CI image saved as /work/